In [ ]:
import demo_util
import numpy as np
import torch
from PIL import Image
import imagenet_classes
from IPython.display import display
import os
from huggingface_hub import hf_hub_download
from modeling.titok import PretrainedTokenizer
from torchvision.utils import save_image


/private/task/jwn/anaconda3/envs/1D_tokeniser/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
torch.backends.cuda.matmul.allow_tf32 = True
torch.manual_seed(0)

In [ ]:
def token_add(tokenizer,vqgan,path,mode,ckpt,save_dir,device):
    original_image = Image.open(path)
    image = torch.from_numpy(np.array(original_image).astype(np.float32)).permute(2, 0, 1).unsqueeze(0) / 255.0
    images=torch.zeros((32,3,256,256))
    tokenizer.init_from_ckpt_de_and_qu(ckpt)
    
    if mode=="stage1":
        z_quantized,_ = tokenizer.encode(image.to(device))
        for i in range(32):
            mask_tokens=z_quantized[:,:,:,0:i]
            decoded_latent = tokenizer.decoder(mask_tokens)
            quantized_states = torch.einsum(
                'nchw,cd->ndhw', decoded_latent.softmax(1),
                vqgan.quantize.embedding.weight)
            reconstructed_image = vqgan.decoder(quantized_states)
            images[i]=reconstructed_image

    if mode=="stage2":
        z_quantized,_ = tokenizer.encode(image.to(device))
        for i in range(32):
            mask_tokens=z_quantized[:,:,:,0:i]
            decoded_latent = tokenizer.decoder(mask_tokens)
            quantized_states = torch.einsum(
                'nchw,cd->ndhw', decoded_latent.softmax(1),
                tokenizer.pixel_quantize.embedding.weight)
            reconstructed_image = tokenizer.pixel_decoder(quantized_states)
            images[i]=reconstructed_image


    images = (images).clamp(0, 1)       
    save_image(images, f"images/{save_dir}.png", nrow=4)



In [ ]:
device = "cuda:2"
config = demo_util.get_config("configs/titok_new.yaml")
titok_tokenizer = demo_util.get_titok_tokenizer_new(config).to(device)
vqgan=PretrainedTokenizer("/private/task/wubin/1d-tokenizer-main/ckpt/maskgit-vqgan-imagenet-f16-256.bin").to(device)

token_add(tokenizer=titok_tokenizer,
          vqgan=vqgan,
          path="assets/ILSVRC2012_val_00008636.png",
          ckpt="",
          save_dir="",
          mode="stage1",
          device=device)